## Simple GenAI app using langchain

In [7]:
# Load data --> Docs --> Splitting data into chunks --> converting chunks into vectors using embeddings --> Vector Store --> Retrieval

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")


In [8]:
# Data Ingestion
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_path="https://docs.smith.langchain.com/administration/tutorials/manage_spend")
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset

In [10]:
# Text Splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)
text_chunks = text_splitter.split_documents(docs)
text_chunks

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing'),
 Document(metadata={'source': 'https://docs.smith.lang

In [9]:
# Embedding 
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [11]:
# Vector Store
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(text_chunks,embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [3]:
# Retrieval Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following questions based on the provided context:
<context>
{context}
</context>
"""
)

document_chain= create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001FCAD223DC0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FCAE746CE0>, root_client=<openai.OpenAI object at 0x000001FCAD223C40>, root_async_client=<openai.AsyncOpenAI object at 0x000001FCAD223C70>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_

In [4]:
# The above document chain first goes through prompt template --> llm model --> StrOutputParser
# returning only the output response to us

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "LangSmith has two usage limits: total traces and extended retention traces",
    "context": [Document(page_content="LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we've been tracking on our usage graph. We can use these in tandem to have granular control over spend.")]
})

#### Above we insert the context manually. However, we want the context to come from the retriever. That way, we can use the retriever to dynamically select the most appropriate document and pass those in for a given question.

In [7]:
# Input --> Retriever --> VectorStoreDB
vectorstoredb

NameError: name 'vectorstoredb' is not defined

In [ ]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(create_retrieval_chain,document_chain)
retrieval_chain

NameError: name 'vectorstoredb' is not defined

In [ ]:
# Get the response from the LLM
response = retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended retention traces"})
response["answer"]